In [ ]:
'''
Jupyter notebook script for configuring radar.

'''

In [1]:
from dca_capture_config import awr1843_config
from mmwave.dataloader.radars import TI


In [2]:
# read the default configuration file 
# take the radar specification inputs and modify the config values accordingly

config_file = '/home/arvind/code/mm-radar/profiles/profile_default.cfg'

config = [line.rstrip('\r\n') for line in open(config_file)]  # use '\r\n' in windows 


In [3]:
# write the configuration files to the radar using serial 
# ignore all commented lines 

# Note: keep chirp time 20% higher than sample duration
# input specifications

range_resolution = 0.025 
max_range = 5
max_velocity = 2
num_chirps = 64
frames_per_second = 10
# num_frames = 0 # infinite

# defaults
# freq_slope_const = 20
start_freq = 77
dig_out_sample_rate = 3200  

# intermediate parameters
wavelength = 3e8 / start_freq / 1e9

# radar config parameters 
num_samples = round(max_range / range_resolution / 0.9)
freq_slope_const = 0.9 * 3e8 * dig_out_sample_rate / 2 / max_range / 1e9
ramp_end_time = round(1.2 * num_samples * dig_out_sample_rate / freq_slope_const / 1e6) # keeping bandwith 20% higher than sampling duration
idle_time = round(wavelength * 1e6 / 4 / max_velocity - ramp_end_time)
frame_periodicity = 1e3 / frames_per_second # frame duration in ms

for index in range(len(config)):
    # Split the line
    split_words = config[index].split(" ")
    # Get the information about the profile configuration
    if "profileCfg" in split_words[0]:
        # split_words[2] = str(start_freq)
        split_words[3] = str(idle_time)
        split_words[5] = str(ramp_end_time)
        split_words[8] = str(freq_slope_const)
        split_words[10] = str(num_samples)
        split_words[11] = str(dig_out_sample_rate)
        config[index] = " ".join(split_words)
        
    # Get the information about the frame configuration    
    if "frameCfg" in split_words[0]:

        # split_words[1] = str(chirp_start_idx)
        # split_words[2] = str(chirp_end_idx)
        split_words[3] = str(num_chirps)
        # split_words[4] = str(num_frames)
        split_words[5] = str(frame_periodicity)
        config[index] = " ".join(split_words)

In [6]:
awr1843 = TI(cli_loc='/dev/ttyS4', data_loc='/dev/ttyS3', num_rx=4, num_tx=1)

config_params = awr1843._initialize(config)
for key, value in config_params.items():
    print(key + '= ' + str(round(value,4)))

awr1843._configure_radar(config)

numDopplerBins= 64.0
numRangeBins= 256
rangeResolutionMeters= 0.025
rangeIdxToMeters= 0.0217
dopplerResolutionMps= 0.0625
maxRange= 5.0
maxVelocity= 2.0001
sensorStop
flushCfg
dfeDataOutputMode 1
channelCfg 15 1 0
adcCfg 2 1
adcbufCfg -1 0 1 1 1
profileCfg 0 77 487 6 0 0 0 86.4 1 222 3200 0 0 30
chirpCfg 0 0 0 0 0 0 0 1
chirpCfg 1 1 0 0 0 0 0 0
frameCfg 0 0 64 0 100.0 1 0
lowPower 0 0
guiMonitor -1 1 1 0 0 0 1
cfarCfg -1 0 2 8 4 3 0 15 1
cfarCfg -1 1 0 4 2 3 1 15 1
multiObjBeamForming -1 1 0.5
clutterRemoval -1 0
calibDcRangeSig -1 0 -5 8 256
extendedMaxVelocity -1 0
lvdsStreamCfg -1 1 1 1
compRangeBiasAndRxChanPhase 0.0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
measureRangeBiasAndRxChanPhase 0 1.5 0.2
CQRxSatMonitor 0 3 5 121 0
CQSigImgMonitor 0 127 4
analogMonitor 0 0
aoaFovCfg -1 -90 90 -90 90
cfarFovCfg -1 0 0 8.92
cfarFovCfg -1 1 -1 1.00
sensorStart
